In [3]:
import csv
import pandas as pd
import numpy as np
import math
import re

# Initilize Columns and Load CSV files

In [4]:
columns = [
  'rowID', 
  'speciesID',
  'common_name',
  'sara_status', 
  'EOO', 
  'IAO',
  'ranges',
  'locations', 
  '_threat_identified',  
]

In [5]:
#read the csv files
df = pd.read_csv('./data-process/CAN-SAR_database.csv',  encoding='ISO-8859-1', index_col=0)
climate_df = pd.read_csv('./data-process/climate.csv')
climate_variables = climate_df.columns[3:]

# Data Processing

In [6]:
filtered_df = df[[col for col in df.columns if any(item in col for item in columns)]].copy()
filtered_df = filtered_df.sort_index()
#append climate_variables to the dataframe
filtered_df = filtered_df.reindex(columns=filtered_df.columns.tolist() + climate_variables.tolist())

In [7]:
#drop the rows with no ranges ie. no locations 
filtered_df.dropna(subset=['ranges'], inplace=True)

In [8]:
#function to append climate data to the dataframe
def append_climate_data(df, id, col = 'ranges'): 
  row = df.loc[id]
  location = row[col].split(' ')
  
  #iterate over the climate variables, get the mean value by list of locations
  for var in climate_variables:
    vals = []
    for loc in location: 
      if loc in climate_df['location'].values:
        vals.append(climate_df[climate_df['location'] == loc][var].values[0])
    
    if vals: 
      df.at[id, var] = np.mean(vals)
    
    if df.at[id, 'locations'] == 'NR': 
      df.at[id, 'locations'] = len(vals)
    
  
for i in filtered_df.index:
  append_climate_data(filtered_df, i)

In [9]:
# drop the rows with no climate data after mapping the climate data
filtered_df.dropna(subset = climate_variables, inplace=True)
#drop the ranges and locations columns since they hold no meaning
filtered_df.drop(['ranges', 'locations'], axis=1, inplace=True)
#replace the NE values with 0, these are the values of the threats. Assume that nan values are 0 meaning no threat
filtered_df.replace('NE', 0, inplace=True)
filtered_df.replace(math.nan, 0, inplace=True)

In [10]:
#convert the EOO and IAO columns from string to float
filtered_df['EOO'] = filtered_df['EOO'].apply(lambda x: float(re.sub("[^0-9]", "", x)) if re.sub("[^0-9]", "", x) else math.nan)
filtered_df['IAO'] = filtered_df['IAO'].apply(lambda x: float(re.sub("[^0-9]", "", x)) if re.sub("[^0-9]", "", x) else math.nan)

In [11]:
#drop the rows with no EOO and IAO values
filtered_df.dropna(subset=['EOO', 'IAO'], inplace=True)

In [12]:
filtered_df

,speciesID,common_name,sara_status,EOO,IAO,X1_threat_identified,X1.1_threat_identified,X1.2_threat_identified,X1.3_threat_identified,X2_threat_identified,...,bio_18,bio_19,bio_2,bio_3,bio_4,bio_5,bio_6,bio_7,bio_8,bio_9
rowID,,,,,,,,,,,,,,,,,,,,,
2,394,allegheny mountain dusky salamander appalachia...,Threatened,562.0,562.0,1,1,1,1,1,...,336.0,138.0,10.537500,21.922981,1324.77230,18.342001,-29.724,48.0660,10.083500,-20.301500
3,396,allegheny mountain dusky salamander carolinian...,Endangered,42.0,42.0,1,0,0,1,0,...,262.0,142.0,13.044084,26.338380,1289.00540,23.987000,-25.538,49.5250,15.322000,-16.376333
10,1501,large-headed woolly yarrow,Special Concern,3112.0,1042.0,1,0,0,0,0,...,233.0,56.0,11.653750,24.392477,1323.87070,22.803000,-24.973,47.7760,15.477000,-14.246500
11,142,athabasca thrift,Special Concern,8382.0,1042.0,1,0,0,0,0,...,233.0,56.0,11.653750,24.392477,1323.87070,22.803000,-24.973,47.7760,15.477000,-14.246500
14,1039,audouin's night-stalking tiger beetle,Threatened,16002.0,362.0,1,1,0,1,1,...,273.0,609.0,9.231999,33.109780,646.25070,16.535000,-11.348,27.8830,-1.914000,6.908667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1028,110,pitcher's thistle,Special Concern,43438.0,136.0,0,0,0,0,0,...,262.0,142.0,13.044084,26.338380,1289.00540,23.987000,-25.538,49.5250,15.322000,-16.376333
1029,359,dwarf lake iris,Special Concern,8232.0,348.0,1,1,0,0,0,...,262.0,142.0,13.044084,26.338380,1289.00540,23.987000,-25.538,49.5250,15.322000,-16.376333
1030,461,buffalograss,Special Concern,2383.0,172.0,1,1,0,0,1,...,229.5,58.5,11.735126,23.363898,1424.60815,22.718500,-27.622,50.3405,14.897417,-17.599417


In [13]:
#map the sara_status to numerical values
Status = {
  'Endangered': 0,
  'Threatened': 1,
  'Special Concern': 2
}

filtered_df['sara_status'] = filtered_df['sara_status'].map(Status)
filtered_df.dropna(subset=['sara_status'], inplace=True)
filtered_df.value_counts('sara_status')

sara_status
0.0    253
2.0    161
1.0    136
dtype: int64

In [14]:
print(list(filtered_df.columns))

['speciesID', 'common_name', 'sara_status', 'EOO', 'IAO', 'X1_threat_identified', 'X1.1_threat_identified', 'X1.2_threat_identified', 'X1.3_threat_identified', 'X2_threat_identified', 'X2.1_threat_identified', 'X2.2_threat_identified', 'X2.3_threat_identified', 'X2.4_threat_identified', 'X3_threat_identified', 'X3.1_threat_identified', 'X3.2_threat_identified', 'X3.3_threat_identified', 'X4_threat_identified', 'X4.1_threat_identified', 'X4.2_threat_identified', 'X4.3_threat_identified', 'X4.4_threat_identified', 'X5_threat_identified', 'X5.1_threat_identified', 'X5.2_threat_identified', 'X5.3_threat_identified', 'X5.4_threat_identified', 'X6_threat_identified', 'X6.1_threat_identified', 'X6.2_threat_identified', 'X6.3_threat_identified', 'X7_threat_identified', 'X7.1_threat_identified', 'X7.2_threat_identified', 'X7.3_threat_identified', 'X8_threat_identified', 'X8.1_threat_identified', 'X8.2_threat_identified', 'X8.3_threat_identified', 'X8.4_threat_identified', 'X8.5_threat_identifie

In [15]:
#drop any remaining nan values
filtered_df.dropna(inplace=True)
filtered_df

,speciesID,common_name,sara_status,EOO,IAO,X1_threat_identified,X1.1_threat_identified,X1.2_threat_identified,X1.3_threat_identified,X2_threat_identified,...,bio_18,bio_19,bio_2,bio_3,bio_4,bio_5,bio_6,bio_7,bio_8,bio_9
rowID,,,,,,,,,,,,,,,,,,,,,
2,394,allegheny mountain dusky salamander appalachia...,1.0,562.0,562.0,1,1,1,1,1,...,336.0,138.0,10.537500,21.922981,1324.77230,18.342001,-29.724,48.0660,10.083500,-20.301500
3,396,allegheny mountain dusky salamander carolinian...,0.0,42.0,42.0,1,0,0,1,0,...,262.0,142.0,13.044084,26.338380,1289.00540,23.987000,-25.538,49.5250,15.322000,-16.376333
10,1501,large-headed woolly yarrow,2.0,3112.0,1042.0,1,0,0,0,0,...,233.0,56.0,11.653750,24.392477,1323.87070,22.803000,-24.973,47.7760,15.477000,-14.246500
11,142,athabasca thrift,2.0,8382.0,1042.0,1,0,0,0,0,...,233.0,56.0,11.653750,24.392477,1323.87070,22.803000,-24.973,47.7760,15.477000,-14.246500
14,1039,audouin's night-stalking tiger beetle,1.0,16002.0,362.0,1,1,0,1,1,...,273.0,609.0,9.231999,33.109780,646.25070,16.535000,-11.348,27.8830,-1.914000,6.908667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1028,110,pitcher's thistle,2.0,43438.0,136.0,0,0,0,0,0,...,262.0,142.0,13.044084,26.338380,1289.00540,23.987000,-25.538,49.5250,15.322000,-16.376333
1029,359,dwarf lake iris,2.0,8232.0,348.0,1,1,0,0,0,...,262.0,142.0,13.044084,26.338380,1289.00540,23.987000,-25.538,49.5250,15.322000,-16.376333
1030,461,buffalograss,2.0,2383.0,172.0,1,1,0,0,1,...,229.5,58.5,11.735126,23.363898,1424.60815,22.718500,-27.622,50.3405,14.897417,-17.599417


# Rename Columns and Save data

In [17]:
#rename the columns to meaningful names
filtered_df.rename(
    columns={
        "bio_1": "Annual Mean Temp",
        "bio_2": "Mean Diurnal Range",
        "bio_3": "Isothermality",
        "bio_4": "Temperature Seasonality",
        "bio_5": "Max Temperature of Warmest Month",
        "bio_6": "Min Temperature of Coldest Month",
        "bio_7": "Temperature Annual Range",
        "bio_8": "Mean Temperature of Wettest Quarter",
        "bio_9": "Mean Temperature of Driest Quarter",
        "bio_10": "Mean Temperature of Warmest Quarter",
        "bio_11": "Mean Temperature of Coldest Quarter",
        "bio_12": "Annual Precipitation",
        "bio_13": "Precipitation of Wettest Month",
        "bio_14": "Precipitation of Driest Month",
        "bio_15": "Precipitation Seasonality",
        "bio_16": "Precipitation of Wettest Quarter",
        "bio_17": "Precipitation of Driest Quarter",
        "bio_18": "Precipitation of Warmest Quarter",
        "bio_19": "Precipitation of Coldest Quarter",
        "EOO": "Estimated Extent of Occurrence",
        "IAO": "Index of Area of Occupancy",
        "X1_threat_identified": "Residential & Commercial Development Threat",
            "X1.1_threat_identified": "Housing & Urban Areas Threat",
            "X1.2_threat_identified": "Commercial & Industrial Areas Threat",
            "X1.3_threat_identified": "Tourism & Recreation Areas Threat",
        "X2_threat_identified": "Agriculture & Aquaculture Threat",
            "X2.1_threat_identified": "Annual & Perennial Non-Timber Crops Threat",
            "X2.2_threat_identified": "Wood & Pulp Plantations Threat",
            "X2.3_threat_identified": "Livestock Farming & Ranching Threat",
            "X2.4_threat_identified": "Marine & Freshwater Aquaculture Threat",
        "X3_threat_identified": "Energy Production & Mining Threat",
            "X3.1_threat_identified": "Oil & Gas Drilling Threat",
            "X3.2_threat_identified": "Mining & Quarrying Threat",
            "X3.3_threat_identified": "Renewable Energy Threat",
        "X4_threat_identified": "Transportation & Service Corridors Threat",
            "X4.1_threat_identified": "Roads & Railroads Threat",
            "X4.2_threat_identified": "Utility & Service Lines Threat",
            "X4.3_threat_identified": "Shipping Lanes Threat",
            "X4.4_threat_identified": "Flight Paths Threat",
        "X5_threat_identified": "Biological Resource Use Threat",
            "X5.1_threat_identified": "Hunting & Collecting Terrestrial Animals Threat",
            "X5.2_threat_identified": "Gathering Terrestrial Plants Threat",
            "X5.3_threat_identified": "Logging & Wood Harvesting Threat",
            "X5.4_threat_identified": "Fishing & Harvesting Aquatic Resources Threat",
        "X6_threat_identified": "Human Intrusions & Disturbance Threat",
            "X6.1_threat_identified": "Recreational Activities Threat",
            "X6.2_threat_identified": "War, Civil Unrest & Military Exercises Threat",
            "X6.3_threat_identified": "Work & Other Activities Threat",
        "X7_threat_identified": "Natural System Modifications Threat",
            "X7.1_threat_identified": "Fire & Fire Suppression Threat",
            "X7.2_threat_identified": "Dams & Water Management/Use Threat",
            "X7.3_threat_identified": "Other Ecosystem Modifications Threat",
        "X8_threat_identified": "Invasive & Other Species & Genes Threat",
            "X8.1_threat_identified": "Invasive Non-Native/Alien Species Threat",
            "X8.2_threat_identified": "Problematic Native Species Threat",
            "X8.3_threat_identified": "Introduced Genetic Material Threat",
        "X9_threat_identified": "Pollution Threat",
            "X9.1_threat_identified": "Household Sewage & Urban Waste Water Threat",
            "X9.2_threat_identified": "Industrial & Military Effluents Threat",
            "X9.3_threat_identified": "Agricultural & Forestry Effluents Threat",
            "X9.4_threat_identified": "Garbage & Solid Waste Threat",
            "X9.5_threat_identified": "Airborne Pollutants Threat",
            "X9.6_threat_identified": "Excess Energy Threat",
        "X10_threat_identified": "Geological Events Threat",
            "X10.1_threat_identified": "Volcanoes Threat",
            "X10.2_threat_identified": "Earthquakes/Tsunamis Threat",
            "X10.3_threat_identified": "Avalanches/Landslides Threat",
        "X11_threat_identified": "Climate Change & Severe Weather Threat",
            "X11.1_threat_identified": "Habitat Shifting & Alteration Threat",
            "X11.2_threat_identified": "Droughts Threat",
            "X11.3_threat_identified": "Temperature Extremes Threat",
            "X11.4_threat_identified": "Storms & Flooding Threat",
    },
    inplace=True)
filtered_df

,speciesID,common_name,sara_status,Estimated Extent of Occurrence,Index of Area of Occupancy,Residential & Commercial Development Threat,Housing & Urban Areas Threat,Commercial & Industrial Areas Threat,Tourism & Recreation Areas Threat,Agriculture & Aquaculture Threat,...,Precipitation of Warmest Quarter,Precipitation of Coldest Quarter,Mean Diurnal Range,Isothermality,Temperature Seasonality,Max Temperature of Warmest Month,Min Temperature of Coldest Month,Temperature Annual Range,Mean Temperature of Wettest Quarter,Mean Temperature of Driest Quarter
rowID,,,,,,,,,,,,,,,,,,,,,
2,394,allegheny mountain dusky salamander appalachia...,1.0,562.0,562.0,1,1,1,1,1,...,336.0,138.0,10.537500,21.922981,1324.77230,18.342001,-29.724,48.0660,10.083500,-20.301500
3,396,allegheny mountain dusky salamander carolinian...,0.0,42.0,42.0,1,0,0,1,0,...,262.0,142.0,13.044084,26.338380,1289.00540,23.987000,-25.538,49.5250,15.322000,-16.376333
10,1501,large-headed woolly yarrow,2.0,3112.0,1042.0,1,0,0,0,0,...,233.0,56.0,11.653750,24.392477,1323.87070,22.803000,-24.973,47.7760,15.477000,-14.246500
11,142,athabasca thrift,2.0,8382.0,1042.0,1,0,0,0,0,...,233.0,56.0,11.653750,24.392477,1323.87070,22.803000,-24.973,47.7760,15.477000,-14.246500
14,1039,audouin's night-stalking tiger beetle,1.0,16002.0,362.0,1,1,0,1,1,...,273.0,609.0,9.231999,33.109780,646.25070,16.535000,-11.348,27.8830,-1.914000,6.908667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1028,110,pitcher's thistle,2.0,43438.0,136.0,0,0,0,0,0,...,262.0,142.0,13.044084,26.338380,1289.00540,23.987000,-25.538,49.5250,15.322000,-16.376333
1029,359,dwarf lake iris,2.0,8232.0,348.0,1,1,0,0,0,...,262.0,142.0,13.044084,26.338380,1289.00540,23.987000,-25.538,49.5250,15.322000,-16.376333
1030,461,buffalograss,2.0,2383.0,172.0,1,1,0,0,1,...,229.5,58.5,11.735126,23.363898,1424.60815,22.718500,-27.622,50.3405,14.897417,-17.599417


In [18]:
# save the dataframe to a csv file
filtered_df.to_csv('final_dataset.csv')